In [ ]:
# Загрузка данных, установка зависимостей
!wget https://lodmedia.hb.bizmrg.com/case_files/815472/train_dataset_train.zip
!wget https://lodmedia.hb.bizmrg.com/case_files/815472/test_dataset_test.zip
!wget https://lodmedia.hb.bizmrg.com/cases/815472/AltGU.zip
!unzip AltGU.zip
!unzip train_dataset_train.zip
!unzip test_dataset_test.zip
!pip install pymorphy2
!pip install googletrans==3.1.0a0
!pip install catboost

In [ ]:
# Ссылка для загрузки данных солнечной активности
https://www.sidc.be/silso/INFO/sndtotcsv.php

In [6]:
import time
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import dbscan,DBSCAN
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, SGDRegressor, RidgeClassifier, LogisticRegressionCV,Ridge,QuantileRegressor,PassiveAggressiveClassifier,PoissonRegressor
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,StackingRegressor,BaggingClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR,SVR,NuSVR
from sklearn.decomposition import TruncatedSVD,PCA,FactorAnalysis,IncrementalPCA,FastICA,KernelPCA,NMF
from sklearn.preprocessing import RobustScaler,QuantileTransformer,PowerTransformer,PolynomialFeatures,KBinsDiscretizer,StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer,MaxAbsScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion,TransformerMixin
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,LocalOutlierFactor
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit,TimeSeriesSplit,GridSearchCV,KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn import set_config
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,roc_auc_score,accuracy_score,f1_score,classification_report
import pymorphy2
from googletrans import Translator
import catboost

In [7]:
# Загрузка данных
employee=pd.read_csv('/content/employees.csv')
trainc=pd.read_csv('/content/train_comments.csv')
traini=pd.read_csv('/content/train_issues.csv')
testc=pd.read_csv('/content/test_comments.csv')
testi=pd.read_csv('/content/test_issues.csv')

# Генерация дополнительных переменных

In [8]:
# Дополнительные переменные времени

traini['created']=pd.to_datetime(traini['created'])
traini['dayofweek']=traini['created'].dt.dayofweek
traini['dayofyear']=traini['created'].dt.dayofyear
traini['year']=traini['created'].dt.year
traini['month']=traini['created'].dt.month
traini['day']=traini['created'].dt.day
traini['hour']=traini['created'].dt.hour

testi['created']=pd.to_datetime(testi['created'])
testi['dayofweek']=testi['created'].dt.dayofweek
testi['dayofyear']=testi['created'].dt.dayofyear
testi['year']=testi['created'].dt.year
testi['month']=testi['created'].dt.month
testi['day']=testi['created'].dt.day
testi['hour']=testi['created'].dt.hour


In [9]:
# Число комментариев
ccdict=dict(zip(trainc['issue_id'].value_counts().index,
                trainc['issue_id'].value_counts()))
ccdict.update(dict(zip(list(set(traini['id'].tolist()).difference(set(trainc['issue_id'].tolist()))),
                 len(set(traini['id'].tolist()).difference(set(trainc['issue_id'].tolist())))*[0])))
traini['ncom']=list(map(lambda x: ccdict[x],traini['id'].tolist()))
ccdictt=dict(zip(testc['issue_id'].value_counts().index,
                testc['issue_id'].value_counts()))
ccdictt.update(dict(zip(list(set(testi['id'].tolist()).difference(set(testc['issue_id'].tolist()))),
                 len(set(testi['id'].tolist()).difference(set(testc['issue_id'].tolist())))*[0])))
testi['ncom']=list(map(lambda x: ccdictt[x],testi['id'].tolist()))

In [ ]:
# Конкатенированные тексты комментариев
tcdict=(dict(zip(trainc['issue_id'].value_counts().index.tolist(),
                [' '.join(trainc['text'][trainc['issue_id']==i].astype('str').str.replace('\n',' ').str.replace(r'(\[.+\])',' ').tolist()) 
                for i in trainc['issue_id'].value_counts().index])))
tcdict.update(dict(zip(list(set(traini['id'].tolist()).difference(set(trainc['issue_id'].tolist()))),
                 len(set(traini['id'].tolist()).difference(set(trainc['issue_id'].tolist())))*[''])))
traini['texts']=list(map(lambda x: tcdict[x],traini['id'].tolist()))
tcdictt=(dict(zip(testc['issue_id'].value_counts().index.tolist(),
                [' '.join(testc['text'][testc['issue_id']==i].astype('str').str.replace('\n',' ').str.replace(r'(\[.+\])',' ').tolist()) 
                for i in testc['issue_id'].value_counts().index])))
tcdictt.update(dict(zip(list(set(testi['id'].tolist()).difference(set(testc['issue_id'].tolist()))),
                 len(set(testi['id'].tolist()).difference(set(testc['issue_id'].tolist())))*[''])))
testi['texts']=list(map(lambda x: tcdictt[x],testi['id'].tolist()))

# Конкатенированные авторы комментариев
tadict=(dict(zip(trainc['issue_id'].value_counts().index.tolist(),
                [' '.join(trainc['author_id'][trainc['issue_id']==i].astype('str').tolist())
                for i in trainc['issue_id'].value_counts().index])))
tadict.update(dict(zip(list(set(traini['id'].tolist()).difference(set(trainc['issue_id'].tolist()))),
                 len(set(traini['id'].tolist()).difference(set(trainc['issue_id'].tolist())))*[''])))
traini['author_ids']=list(map(lambda x: tadict[x],traini['id'].tolist()))
tadictt=(dict(zip(testc['issue_id'].value_counts().index.tolist(),
                [' '.join(testc['author_id'][testc['issue_id']==i].astype('str').tolist())
                for i in testc['issue_id'].value_counts().index])))
tadictt.update(dict(zip(list(set(testi['id'].tolist()).difference(set(testc['issue_id'].tolist()))),
                 len(set(testi['id'].tolist()).difference(set(testc['issue_id'].tolist())))*[''])))
testi['author_ids']=list(map(lambda x: tadictt[x],testi['id'].tolist()))

In [11]:
# Внесение дополнительной информации о работниках
edict=dict(zip(employee['id'],employee[['active', 'full_name', 'position', 'hiring_type', 'payment_type',
       'passport', 'is_nda_signed',
       'is_labor_contract_signed', 'is_added_to_internal_chats',
       'is_added_one_to_one']].values))
for cnt1 in ['assignee_id','creator_id']:
  for cnt2 in list(enumerate(['active', 'full_name', 'position', 'hiring_type', 'payment_type',
       'passport', 'is_nda_signed',
       'is_labor_contract_signed', 'is_added_to_internal_chats',
       'is_added_one_to_one'])):
    traini[cnt1+'_'+cnt2[1]]=np.array(list(map(lambda x: edict[x],traini['assignee_id'].tolist())))[:,cnt2[0]]
for cnt1 in ['assignee_id','creator_id']:
  for cnt2 in list(enumerate(['active', 'full_name', 'position', 'hiring_type', 'payment_type',
       'passport', 'is_nda_signed',
       'is_labor_contract_signed', 'is_added_to_internal_chats',
       'is_added_one_to_one'])):
    testi[cnt1+'_'+cnt2[1]]=np.array(list(map(lambda x: edict[x],testi['assignee_id'].tolist())))[:,cnt2[0]]

In [ ]:
# Леммтизация
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text): 
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return ' '.join(res)
traini['summary_l']=list(map(lemmatize, traini['summary'].tolist()))
testi['summary_l']=list(map(lemmatize, testi['summary'].tolist()))
traini['texts_l']=list(map(lemmatize, traini['texts'].tolist()))
testi['texts_l']=list(map(lemmatize, testi['texts'].tolist()))

In [ ]:
translator = Translator(service_urls=['translate.googleapis.com'])
def langdet (x):
  lang=translator.detect(x).lang
  if type(lang)==list:
    lang=' '.join(lang)
  return lang
# Детекция языка резюме и комментариев
traini['summary_lang'] = list(map(langdet, traini['summary'].astype('str').tolist()))
traini['texts_lang'] = list(map(langdet, traini['texts'].astype('str').tolist()))
testi['texts_lang'] = list(map( langdet, testi['texts'].astype('str').tolist()))
testi['summary_lang'] = list(map( langdet, testi['summary'].astype('str').tolist()))
# Перевод на английский языка резюме и комментариев
traini['summary_tr']=list(map( lambda x: translator.translate(x,dest='en').text, traini['summary'].astype('str').tolist()))
traini['texts_tr']=list(map( lambda x: translator.translate(x,dest='en').text, traini['texts'].astype('str').tolist()))
testi['summary_tr']=list(map( lambda x: translator.translate(x,dest='en').text, testi['summary'].astype('str').tolist()))
testi['texts_tr']=list(map( lambda x: translator.translate(x,dest='en').text, testi['texts'].astype('str').tolist()))

In [13]:
# Внесение данных солнечной активности
trainis=traini[['year','month','day','created']].copy()
trainis['ind']=trainis.index.tolist()
trainis.index=traini['created']
trainis=trainis.sort_index()
trainis.index=trainis['year'].astype('str')+':'+trainis['month'].astype('str')+':'+trainis['day'].astype('str')
testis=testi[['year','month','day','created']].copy()
testis['ind']=testis.index.tolist()
testis.index=testi['created']
testis=testis.sort_index()
testis.index=testis['year'].astype('str')+':'+testis['month'].astype('str')+':'+testis['day'].astype('str')

sunact=pd.read_csv('/content/drive/MyDrive/Russia3/SN_d_tot_V2.0.csv',sep=';')
sunact.columns=['year','month','day','_','sunspots','sunspots2','sunspots3','sunspots4']
sunacts=sunact[['sunspots','sunspots2','sunspots3']]
sunacts.index=sunact['year'].astype('str')+':'+sunact['month'].astype('str')+':'+sunact['day'].astype('str')
for cnt in ['sunspots','sunspots2','sunspots3']:
  trainis[cnt]=sunacts[cnt]
  testis[cnt]=sunacts[cnt]
trainis.index=trainis['ind']
trainis=trainis.sort_index()
testis.index=testis['ind']
testis=testis.sort_index()
for cnt in ['sunspots3']:
  traini[cnt]=trainis[cnt].tolist()
for cnt in ['sunspots3']:
  testi[cnt]=testis[cnt].tolist()

In [14]:
# Внесение дня и генерация данных линейной регрессии таргета по дню (временной тренд)
traini['days']=(traini['created']-traini['created'].min()).dt.days
testi['days']=(testi['created']-traini['created'].min()).dt.days

input1=Pipeline([('input',ColumnTransformer([
              ('text',CountVectorizer(ngram_range = (1,1),max_df=0.9, min_df=2),'summary'),
              ('text2',CountVectorizer(ngram_range = (1,1),max_df=0.9, min_df=2),'author_ids'),
              ('onehot',OneHotEncoder(handle_unknown='ignore'),['project_id', 'assignee_id',
       'creator_id'])
               ])),
              ('tfidf',TfidfTransformer())])

pipe=Pipeline([('input',ColumnTransformer([
              ('inp1',input1,['summary','author_ids','project_id', 'assignee_id','creator_id']),
              ('scale',StandardScaler(),['days'])])),
              ('rgs',LinearSVR(C=10000,loss='epsilon_insensitive',random_state=0,max_iter=10000))])
pipe2=Pipeline([('input',ColumnTransformer([
              ('inp1',input1,['summary','author_ids','project_id', 'assignee_id','creator_id']),
              ('scale',StandardScaler(),['days'])])),
              ('rgs',LinearSVR(C=0.1,loss='squared_epsilon_insensitive',random_state=0,max_iter=10000))])

traini['time']=pipe.fit(traini[traini['overall_worklogs']<1e6],traini['overall_worklogs'][traini['overall_worklogs']<1e6],
                        rgs__sample_weight=np.power(traini['overall_worklogs'][traini['overall_worklogs']<1e6],0.4)/
                        np.power(traini['overall_worklogs'][traini['overall_worklogs']<1e6],0.4).mean()).predict(traini)
testi['time']=pipe.predict(testi)
traini['time_log']=pipe2.fit(traini[traini['overall_worklogs']<1e6],np.log(traini['overall_worklogs'][traini['overall_worklogs']<1e6]),
                        rgs__sample_weight=np.power(traini['overall_worklogs'][traini['overall_worklogs']<1e6],0.4)/
                        np.power(traini['overall_worklogs'][traini['overall_worklogs']<1e6],0.4).mean()).predict(traini)
testi['time_log']=pipe2.predict(testi)

# Обучение модели

In [ ]:
# Начальное обучение catboost регрессора

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(
       pd.concat((traini[['project_id', 
       'creator_id', 'dayofweek', 'dayofyear', 'year', 'month', 'day',
       'hour', 'assignee_id_active', 'assignee_id_full_name', 'assignee_id_position',
       'assignee_id_hiring_type', 'assignee_id_payment_type',
       'assignee_id_passport', 'assignee_id_is_nda_signed',
       'creator_id_full_name', 'creator_id_position', 'creator_id_hiring_type',
       'creator_id_payment_type', 'creator_id_passport',
       'creator_id_is_nda_signed', 
       'creator_id_is_added_to_internal_chats', 'ncom', 'sunspots3','days','time', 'time_log']].fillna(-1),
       pd.DataFrame(np.hstack((CountVectorizer(min_df=2).fit_transform(traini['summary'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit_transform(traini['summary_l'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 1),min_df=2).fit_transform(traini['summary_tr'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char", min_df=2).fit_transform(traini['key'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char_wb",min_df=2).fit_transform(traini['assignee_id_full_name'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char_wb",min_df=2).fit_transform(traini['creator_id_full_name'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit_transform(traini['texts'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit_transform(traini['texts_l'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 1),min_df=2).fit_transform(traini['texts_tr'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 1),token_pattern=r"(?u)\b\w+\b",min_df=2).fit_transform(traini['author_ids'].astype('str')).todense())))),axis=1)[traini['overall_worklogs']<1e6],
       traini['overall_worklogs'][traini['overall_worklogs']<1e6],test_size=0.1,random_state=0)

In [ ]:
rgs=catboost.CatBoostRegressor(iterations=2000,
                          learning_rate=0.05,loss_function='MAE',
                          eval_metric='R2', bagging_temperature=150)

In [ ]:
rgs.fit(X_train,y_train,
        cat_features=['project_id', 
       'creator_id', 'dayofweek', 'dayofyear', 'year', 'month', 'day',
       'hour', 'assignee_id_active', 'assignee_id_full_name', 'assignee_id_position',
       'assignee_id_hiring_type', 'assignee_id_payment_type',
       'assignee_id_passport', 'assignee_id_is_nda_signed',
       'creator_id_full_name', 'creator_id_position', 'creator_id_hiring_type',
       'creator_id_payment_type', 'creator_id_passport',
       'creator_id_is_nda_signed', 
       'creator_id_is_added_to_internal_chats'],
        eval_set=(X_val,y_val),verbose=50)

In [ ]:
# Выявление значимых переменных
filter=rgs.feature_importances_!=0

In [ ]:
# Финальное обучение catboost регрессора

In [ ]:
rgs=catboost.CatBoostRegressor(iterations=5000,
                          learning_rate=0.05,loss_function='MAE',
                          eval_metric='R2', bagging_temperature=150)

In [ ]:
rgs.fit(pd.concat((traini[['project_id', 
       'creator_id', 'dayofweek', 'dayofyear', 'year', 'month', 'day',
       'hour', 'assignee_id_active', 'assignee_id_full_name', 'assignee_id_position',
       'assignee_id_hiring_type', 'assignee_id_payment_type',
       'assignee_id_passport', 'assignee_id_is_nda_signed',
       'creator_id_full_name', 'creator_id_position', 'creator_id_hiring_type',
       'creator_id_payment_type', 'creator_id_passport',
       'creator_id_is_nda_signed', 'creator_id_is_added_to_internal_chats', 
       'ncom', 'sunspots3','days','time', 'time_log']].fillna(-1),
       pd.DataFrame(np.hstack((CountVectorizer(min_df=2).fit_transform(traini['summary'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit_transform(traini['summary_l'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 1),min_df=2).fit_transform(traini['summary_tr'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char", min_df=2).fit_transform(traini['key'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char_wb",min_df=2).fit_transform(traini['assignee_id_full_name'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char_wb",min_df=2).fit_transform(traini['creator_id_full_name'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit_transform(traini['texts'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit_transform(traini['texts_l'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 1),min_df=2).fit_transform(traini['texts_tr'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 1),token_pattern=r"(?u)\b\w+\b",min_df=2).fit_transform(traini['author_ids'].astype('str')).todense())))),axis=1)[traini['overall_worklogs']<1e6].iloc[:,filter],
       traini['overall_worklogs'][traini['overall_worklogs']<1e6],
        cat_features=['project_id', 
       'creator_id', 'dayofweek', 'dayofyear', 'year', 'month', 'day',
       'hour', 'assignee_id_active', 'assignee_id_full_name', 'assignee_id_position',
       'assignee_id_hiring_type', 'assignee_id_payment_type',
       'creator_id_full_name', 'creator_id_position', 'creator_id_hiring_type',
       'creator_id_payment_type', 'creator_id_passport'],
        sample_weight=(np.cbrt(traini['overall_worklogs'][traini['overall_worklogs']<1e6])/
        np.cbrt(traini['overall_worklogs'][traini['overall_worklogs']<1e6]).mean()),verbose=50)

In [ ]:
#Прогноз
testi['overall_worklogs']=rgs.predict(pd.concat((testi[['project_id', 
       'creator_id', 'dayofweek', 'dayofyear', 'year', 'month', 'day',
       'hour', 'assignee_id_active', 'assignee_id_full_name', 'assignee_id_position',
       'assignee_id_hiring_type', 'assignee_id_payment_type',
       'assignee_id_passport', 'assignee_id_is_nda_signed',
       'creator_id_full_name', 'creator_id_position', 'creator_id_hiring_type',
       'creator_id_payment_type', 'creator_id_passport',
       'creator_id_is_nda_signed', 'creator_id_is_added_to_internal_chats', 
       'ncom', 'sunspots3','days','time', 'time_log']].fillna(-1),
       pd.DataFrame(np.hstack((CountVectorizer(min_df=2).fit(traini['summary'].astype('str')).transform(testi['summary'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit(traini['summary_l'].astype('str')).transform(testi['summary_l'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit(traini['summary_tr'].astype('str')).transform(testi['summary_tr'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char", min_df=2).fit(traini['key'].astype('str')).transform(testi['key'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char_wb",min_df=2).fit(traini['assignee_id_full_name'].astype('str')).transform(testi['assignee_id_full_name'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 4),analyzer="char_wb",min_df=2).fit(traini['creator_id_full_name'].astype('str')).transform(testi['creator_id_full_name'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit(traini['texts'].astype('str')).transform(testi['texts'].astype('str')).todense(),                    
                    CountVectorizer(min_df=2).fit(traini['texts_l'].astype('str')).transform(testi['texts_l'].astype('str')).todense(),
                    CountVectorizer(min_df=2).fit(traini['texts_tr'].astype('str')).transform(testi['texts_tr'].astype('str')).todense(),
                    CountVectorizer(ngram_range=(1, 1),token_pattern=r"(?u)\b\w+\b",min_df=2).fit(traini['author_ids'].astype('str')).transform(testi['author_ids'].astype('str')).todense())))),axis=1).iloc[:,filter])
testi['overall_worklogs']=testi['overall_worklogs'].astype('int')
submit=testi[['id','overall_worklogs']]
submit.to_csv('submit.csv',index=False)